In [ ]:
import numpy as np
import tensorflow as tf
from keras.applications.vgg19 import VGG19, preprocess_input
from keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Flatten, Dropout
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping


In [ ]:
# Define the data directories
train_dir = '../Data/Train/'
valid_dir = '../Data/Test/'

# Set the image size and batch size
img_size = (255, 255)
batch_size = 3


In [ ]:
# Define the data generators for training and validation
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical'
)

valid_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rescale=1./255
)
valid_generator = valid_datagen.flow_from_directory(
    valid_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical'
)


In [ ]:
# Load the pre-trained VGG19 model
model = VGG19(
    weights='imagenet',
    include_top=False,
    input_shape=(img_size[0], img_size[1], 3)
)

# Freeze the pre-trained layers
for layer in model.layers:
    layer.trainable = False

# Add new classification layers
x = model.output
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(train_generator.num_classes, activation='softmax')(x)

# Define model
model = Model(
    inputs=model.input,
    outputs=predictions
)

# Compile the model
model.compile(
    optimizer=Adam(lr=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Set up the EarlyStopping callback
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=3
)


In [ ]:
# Train the model
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=valid_generator,    
    callbacks=[early_stopping]
)

In [ ]:
# Save the trained model
model.save('my_model.h5')


In [ ]:
import tensorflow as tf

model = load_model('my_model.h5')

image_path = '../Data/Test/Green_Good/46_flr.jpg'
image = tf.io.read_file(image_path)
image = tf.image.decode_image(image, channels=3)
image = tf.expand_dims(image, axis=0)
image = tf.image.resize(image, size=(img_size[0], img_size[1]))
x = preprocess_input(image)

preds = model.predict(x)

class_idx = np.argmax(preds[0])
class_label = train_generator.class_indices
for k, v in class_label.items():
    if v == class_idx:
        print(f"Predicted class: {k}")
        break
